In [1]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana import system_program
from solana import transaction
from solana.system_program import SYS_PROGRAM_ID, CreateAccountParams, create_account
from solana.transaction import *
from spl.token.constants import TOKEN_PROGRAM_ID, ASSOCIATED_TOKEN_PROGRAM_ID
from spl.token.instructions import initialize_mint, InitializeMintParams
from spl.token._layouts import MINT_LAYOUT
from spl.token.client import Token
from spl.token import instructions
import base64
import base58
from borsh_construct import *
from solana.rpc.commitment import Commitment
import time



client = Client("https://api.devnet.solana.com")

client.is_connected()

True

In [2]:
program_id = PublicKey("2dJFaB4Zz7dHKHWvLqwS27M5ma1p3TrCNZutSiDJoFGg")
metadata_program_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")


In [3]:
# client.request_airdrop(PublicKey("DxkiJXK31VEd4R6F44mQRszoUQugpPYMwL9ybcrnt22r"), 2000000000)

In [4]:
accountSchema = CStruct(
    "vault_total" / F32,
    "counter" / U32,
)

request_units = CStruct(
    "units" / U32,
    "additional_fee" / U32   
)

def getAccountInfo(client: Client, account_pk: PublicKey):
    """Fetch account information from RPC, parse out the data and deserialize"""
    comm = Commitment("confirmed")
    res = client.get_account_info(account_pk, comm, encoding='base64')
    data = res['result']
    if isinstance(data, dict):
        return accountSchema.parse(base64.urlsafe_b64decode(data['value']['data'][0]))
    else:
        raise AttributeError(f'Unknown RPC result {data}')

In [5]:
seed = bytes([125,215,212,54,213,98,45,123]*4)
payer_keypair = Keypair.from_seed(seed)
client.request_airdrop(payer_keypair.public_key, 2*10**9)
base58.b58encode(payer_keypair.secret_key)

b'3Wvr1y8TQRVBw4DiyE4j9Lu8s1ycDYfBasNcNK5TSpDzzhSAZK712J89yevZUUAbRsyC3c9V13giG9ojJU4MbQ1p'

In [6]:
dsol_mint = PublicKey("4hnebtBZBkWF8NvPr4XUdKQFNj2tHim8nqephcMCsBos")
governor_mint = PublicKey("DeEoeaJQctx5VxJzKbt7iHoZ4rjdDmhwfQSnHVUoWokC")

payer_governor_token_address = instructions.get_associated_token_address(payer_keypair.public_key, governor_mint)

table_mint_keypair = Keypair.generate()

payer_table_associated_token_address = instructions.get_associated_token_address(payer_keypair.public_key, table_mint_keypair.public_key)

(payer_table_data_pda, payer_table_data_pda_bump) = PublicKey.find_program_address([b"table_data_pda", bytes(table_mint_keypair.public_key)], program_id)

(payer_table_metadata_pda, payer_table_metadata_pda_bump) = PublicKey.find_program_address([b"metadata", bytes(metadata_program_pubkey), bytes(table_mint_keypair.public_key)], metadata_program_pubkey) 

(table_sales_pda, table_sales_nonce) = PublicKey.find_program_address([b'table_sales_pda'], program_id)

payer_dsol_token_account = instructions.get_associated_token_address(payer_keypair.public_key, dsol_mint)

(table_mint_authority, nonce) = PublicKey.find_program_address([b"table_mint_authority_pda"], program_id)



In [7]:
seed = bytes([25,215,212,154,213,198,45,123]*4)
vault_keypair = Keypair.from_seed(seed)
# vault_keypair = Keypair.generate()
# client.request_airdrop(vault_keypair.public_key, 2*10**9)
# time.sleep(5)
vault_dsol_token_account = instructions.get_associated_token_address(vault_keypair.public_key, dsol_mint)

In [8]:

payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
payer_dsol_token_account_meta = AccountMeta(payer_dsol_token_account, False, True)
payer_governor_token_account_meta = AccountMeta(payer_governor_token_address, False, True)
payer_table_associated_account_meta = AccountMeta(payer_table_associated_token_address, False, True)
payer_table_data_pda_meta = AccountMeta(payer_table_data_pda, False, True)
payer_table_metadata_meta = AccountMeta(payer_table_metadata_pda, False, True)
vault_account_meta = AccountMeta(vault_keypair.public_key, False, True)
vault_dsol_token_account_meta = AccountMeta(vault_dsol_token_account, False, True)
table_mint_account_meta = AccountMeta(table_mint_keypair.public_key, True, True)
table_mint_authority_meta = AccountMeta(table_mint_authority, False, False)
table_sales_pda_meta = AccountMeta(table_sales_pda, False, False)

rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
spl_program_meta  = AccountMeta(TOKEN_PROGRAM_ID, False, False)
sys_program_meta = AccountMeta(SYS_PROGRAM_ID, False, False)
sysvar_clock_meta = AccountMeta(solana.sysvar.SYSVAR_CLOCK_PUBKEY, False, False)
associated_program_meta = AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False)
metadata_meta = AccountMeta(metadata_program_pubkey, False, False)
dsol_mint_meta = AccountMeta(dsol_mint, False, False)
compute_program_meta = AccountMeta(PublicKey("ComputeBudget111111111111111111111111111111"), False, False)



In [9]:


print(dsol_mint)
print(vault_keypair.public_key)
print(vault_dsol_token_account)
print(payer_keypair.public_key)
print(payer_dsol_token_account)
print(payer_governor_token_address)
print(sys_program_meta)
print(spl_program_meta)
print(payer_table_metadata_pda)

4hnebtBZBkWF8NvPr4XUdKQFNj2tHim8nqephcMCsBos
3HqXocfan3tVv3N4tTAFhyVePkP856jxngQ6pQxdupXP
9ZJkPCjEZehchPHs1k327BCb9JQVEg7drn7nmkK6Wkqk
2ASw3tjK5bSxQxFEMsM6J3DnBozNh7drVErSwc7AtzJv
4JTdYhzx6rorDjPMGdXDQCLuF3nM8dpiodnUgqqqhHEc
6MZNR1CdWYhiLEntkN7bcyQYkWy53URX7sJRZaUoTWkH
AccountMeta(pubkey=11111111111111111111111111111111, is_signer=False, is_writable=False)
AccountMeta(pubkey=TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA, is_signer=False, is_writable=False)
41Qi3PX3QYFbhtvgjR3NdQLgUe4JmVMpByYsqNUKHLoF


In [10]:
def init_table_mint():
    accounts = [
        payer_account_meta,
        payer_dsol_token_account_meta,
        payer_governor_token_account_meta,
        payer_table_associated_account_meta,
        payer_table_data_pda_meta,
        payer_table_metadata_meta,
        vault_account_meta,
        vault_dsol_token_account_meta,
        table_mint_account_meta,
        table_mint_authority_meta,
        table_sales_pda_meta,
        sys_program_meta,
        spl_program_meta,
        rent_account_meta,
        sysvar_clock_meta,

        # compute_program_meta,
        spl_program_meta,
        sys_program_meta,
        spl_program_meta,
        associated_program_meta,
        spl_program_meta,
        metadata_meta,
        metadata_meta,
        spl_program_meta,
        spl_program_meta,
        sys_program_meta,
    ]


    transaction = Transaction()

    request_units = CStruct(
    "units" / U32,
    "additional_fee" / U32   
    )


    transaction.add(
        TransactionInstruction(
            [],
            compute_program_meta.pubkey,
            request_units.build({"units": 500000, "additional_fee": int(0.01*10**9)})
        )
    )

    transaction.add(
        TransactionInstruction(
            accounts,
            program_id,
            bytes([0, 10])
        )
    )
    
    return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, table_mint_keypair)['result']}?cluster=devnet"

init_table_mint()

# time.sleep(5)

RPCException: {'code': -32002, 'message': 'Transaction simulation failed: Error processing Instruction 1: Program failed to complete', 'data': {'accounts': None, 'err': {'InstructionError': [1, 'ProgramFailedToComplete']}, 'logs': ['Program 2dJFaB4Zz7dHKHWvLqwS27M5ma1p3TrCNZutSiDJoFGg invoke [1]', 'Program log: MintTable { governor_reward: 10 }', 'Program log: Position 6MZNR1CdWYhiLEntkN7bcyQYkWy53URX7sJRZaUoTWkH', 'Program log: Position 2', 'Program log: Position NM7Wmv7GQBsWYTvz5X6KuhLvE4N51AfCjoee3vsTtmV NM7Wmv7GQBsWYTvz5X6KuhLvE4N51AfCjoee3vsTtmV', 'Program log: Current timestamp: 1645872457', 'Program log: Transfer DSOL token', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA invoke [2]', 'Program log: Instruction: Transfer', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA consumed 2643 of 142652 compute units', 'Program TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA success', 'Program log: Hello', 'Program 11111111111111111111111111111111 invoke [2]', 'Program 1111111111111

In [ ]:
program_id

CrznSBPyrfdxzzzBsRmvm2R1ZeFJz5aDzSpbutGSH3Mb

In [ ]:
# def mint_avatar():
#     accounts = [
#         # program_id_account_meta,
#         payer_account_meta,
#         payer_dsol_token_account_meta,
#         vault_account_meta, 
#         vault_dsol_token_account_meta,
#         mint_account_meta, 
#         rent_account_meta, 
#         associated_account_meta,
#         spl_program_meta,
#         AccountMeta(SYS_PROGRAM_ID, False, False),
#         mint_authority_meta,
#         avatar_data_pda_meta,
#         sales_pda_meta,
#         AccountMeta(metadata_pda, False, True),
#         sysvar_clock_meta,

#         spl_program_meta,
#         AccountMeta(ASSOCIATED_TOKEN_PROGRAM_ID, False, False),
#         spl_program_meta,
#         spl_program_meta,
#         AccountMeta(metadata_program_pubkey, False, False),
#     ]   

#     transaction = Transaction()
#     transaction.add(TransactionInstruction(
#         accounts,
#         program_id,
#         bytes([0])
#     ))
    
#     return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account, vault_keypair)['result']}?cluster=devnet"

# mint_avatar()

'https://explorer.solana.com/tx/5tLuzXr1cT9Bxx12DJc7wzMLRT6KC4Wp9J16xicTpTbgYsjqJVNg9fARSVY3h1EYgqeNEYutGXyGh8wfesFptEep?cluster=devnet'

In [ ]:

# accountSchema = CStruct(
#     "vault_total" / F32,
#     "counter" / U32,
# )

# (sales_pda, sales_nonce) = PublicKey.find_program_address([b'sales_pda', bytes(program_id)], program_id)
# print(f"PDA: {sales_pda, sales_nonce}")

# getAccountInfo(client,sales_pda)

PDA: (ALXB7w5iEnonUTum1aF3f2W6rJJrgL94iEEv2bixTpez, 253)


Container(vault_total=57.16346740722656, counter=28)

In [ ]:
# accountSchema = CStruct(
#     "date_created" / U32,
#     "unlockable_date" / U32,
#     "numeration" /U32,
#     "rarity" / U8,
#     "level" / U8,
#     "xp" / U32,
# )


# (locktime_pda, locktime_nonce) = PublicKey.find_program_address([b'locktime_pda', bytes(mint_account_meta.pubkey), bytes(associated_account_meta.pubkey)], program_id)
# print(f"PDA: {avatar_data_pda, avatar_pda_nonce}")

# getAccountInfo(client,avatar_data_pda)

PDA: (BvggzwsB7fm3D5DDEJ8d8i4DkpDBcEWtTkzB3DCxgKDR, 255)


Container(date_created=1643887151, unlockable_date=1643887240, numeration=28, rarity=1, level=0, xp=0)

In [ ]:
# time.sleep(120)

# new_accounts = [
#     mint_account_meta,
#     associated_account_meta,
#     spl_program_meta,
#     mint_authority_meta,
#     avatar_data_pda_meta,
#     sysvar_clock_meta,

#     spl_program_meta
# ]

# transaction = Transaction()
# transaction.add(TransactionInstruction(
#     new_accounts,
#     program_id,
#     bytes([1])
# ))
# f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair, mint_account)['result']}?cluster=devnet"

'https://explorer.solana.com/tx/5eptcFoXcqNqZVCRnezsRGuEYWUhsPzoMN3Uj1KUePSCquZNC9gWwQkBtqH5gfhWFNnJyXMWBYNpZ1PQS8Es4yvu?cluster=devnet'

b' \xa1\x07\x00\x80\x96\x98\x00'